In [1]:
# !pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=fdf71da77a2004859e74da145d6f0e937e558c1df4051e01075a4e75881c2679
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [166]:
# import the necessary libraries/modules
import ta
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [167]:
# constants
stop_loss_multiplier = 1 # change between 1.0 and 1.5
risk_reward_ratio = 2.0
# Calculate the ATR (used to determine stop loss distance)
atr_period = 14
adx_period = 14  # Common period for ADX is 14

In [168]:
df = pd.read_csv('BTCUSD_M15_TRAIN.csv')
len(df)

60000

In [169]:
def data_cleaning(df):
  '''
  This function takes in a dataframe that must have the following columns: Open, High, Low, Close.
  '''
  if 'Vol' in df.columns:
    del df['Vol']
  if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y %H:%M')
    df.set_index('Date', inplace=True)
  df['sma_6'] = ta.trend.SMAIndicator(df['Close'], window=6).sma_indicator()
  df['sma_20'] = ta.trend.SMAIndicator(df['Close'], window=20).sma_indicator()
  df['sma_100'] = ta.trend.SMAIndicator(df['Close'], window=100).sma_indicator()
  df['adx'] = ta.trend.ADXIndicator(high=df['High'], low=df['Low'], close=df['Close'], window=adx_period).adx()
  df['bb_width'] = ta.volatility.BollingerBands(df['Close'], window=20, window_dev=2).bollinger_wband()
  df['bb_pband'] = ta.volatility.BollingerBands(df['Close'], window=20, window_dev=2).bollinger_pband()
  df['trend'] = np.where(df['Close'] > df['sma_100'], 1, 2)
# Check for crossover (SMA-6 crosses above SMA-20)
  df['crossover'] = ((df['sma_6'] > df['sma_20']) & (df['sma_6'].shift(1) <= df['sma_20'].shift(1))).astype(int)

# Check for crossunder (SMA-6 crosses below SMA-20)
  df['crossunder'] = ((df['sma_6'] < df['sma_20']) & (df['sma_6'].shift(1) >= df['sma_20'].shift(1))).astype(int)

# Only consider crossovers/crossunders in the direction of the trend
  df['valid_cross'] = np.where(df['crossover'],1,
                             np.where(df['crossunder'],2,3))
  df['ATR'] = ta.volatility.AverageTrueRange(df['High'], df['Low'], df['Close'], window=atr_period).average_true_range()
# Calculate initial stop loss and take profit levels
  df['LowerStopLoss'] = df['Low'] - (df['ATR'] * stop_loss_multiplier)
  df['UpperStopLoss'] = df['High'] + (df['ATR'] * stop_loss_multiplier)

  # Calculate the take profit based on risk-reward ratio (2x the distance from stop loss)
  df['LowerTakeProfit'] = df['Close'] - ((df['UpperStopLoss'] - df['Close']) * risk_reward_ratio)
  df['UpperTakeProfit'] = df['Close'] + ((df['Close'] - df['LowerStopLoss']) * risk_reward_ratio)
  df.dropna(inplace=True)

  return df

In [170]:
def evaluate_trades_precisely(df):
    df['trade_result'] = 3
    # df['trade_duration'] = 0
    i = 0

    while i < len(df):
        if df['valid_cross'].iloc[i] == 1:  # Long trade signal
            entry_price = df['Close'].iloc[i]
            stop_loss = df['LowerStopLoss'].iloc[i]
            take_profit = df['UpperTakeProfit'].iloc[i]

            for j in range(i+1, len(df)):
                if df['Low'].iloc[j] <= stop_loss:
                    df.loc[df.index[i], 'trade_result'] = 3  # Stop loss hit
                    # df.loc[df.index[i], 'trade_duration'] = j - i
                    # i = j  # Move outer loop to this point
                    break
                elif df['High'].iloc[j] >= take_profit:
                    df.loc[df.index[i], 'trade_result'] = 1  # Take profit hit
                    # df.loc[df.index[i], 'trade_duration'] = j - i
                    # i = j  # Move outer loop to this point
                    break
            else:
                # Trade didn't conclude within available data
                df.loc[df.index[i], 'trade_result'] = -1
                # df.loc[df.index[i], 'trade_duration'] = len(df) - i - 1

        elif df['valid_cross'].iloc[i] == 2:  # Short trade signal
            entry_price = df['Close'].iloc[i]
            stop_loss = df['UpperStopLoss'].iloc[i]
            take_profit = df['LowerTakeProfit'].iloc[i]

            for j in range(i+1, len(df)):
                if df['High'].iloc[j] >= stop_loss:
                    df.loc[df.index[i], 'trade_result'] = 3  # Stop loss hit
                    # df.loc[df.index[i], 'trade_duration'] = j - i
                    # i = j  # Move outer loop to this point to avoid repeating trades
                    break
                elif df['Low'].iloc[j] <= take_profit:
                    df.loc[df.index[i], 'trade_result'] = 2  # Take profit hit
                    # df.loc[df.index[i], 'trade_duration'] = j - i
                    # i = j  # Move outer loop to this point to avoid repeating trades
                    break
            else:
                # Trade didn't conclude within available data
                df.loc[df.index[i], 'trade_result'] = -1
                # df.loc[df.index[i], 'trade_duration'] = len(df) - i - 1

        i += 1  # Move to next candle if no trade was initiated

    return df

In [171]:
df = data_cleaning(df)
df = evaluate_trades_precisely(df)

In [172]:
df.head(20)

,Open,High,Low,Close,sma_6,sma_20,sma_100,adx,bb_width,bb_pband,trend,crossover,crossunder,valid_cross,ATR,LowerStopLoss,UpperStopLoss,LowerTakeProfit,UpperTakeProfit,trade_result
Date,,,,,,,,,,,,,,,,,,,,
2021-10-07 06:30:00,54691.4,54691.4,54492.9,54498.1,54667.016667,54848.090,53773.157,13.196027,1.248902,-0.010935,1,0,0,3,250.025805,54242.874195,54941.425805,53611.448390,55008.551610,3
2021-10-07 06:45:00,54496.9,54622.4,54364.4,54488.3,54630.333333,54835.450,53803.218,13.608975,1.365987,0.036543,1,0,0,3,250.595390,54113.804610,54872.995390,53718.909219,55237.290781,3
2021-10-07 07:00:00,54488.3,54552.6,54228.2,54291.0,54575.150000,54805.570,53831.156,14.481079,1.613190,-0.082015,1,0,0,3,255.867148,53972.332852,54808.467148,53256.065704,54928.334296,3
2021-10-07 07:15:00,54287.7,54563.4,54269.3,54547.3,54551.333333,54780.600,53860.645,15.226554,1.610692,0.235592,1,0,0,3,258.598066,54010.701934,54821.998066,53997.903868,55620.496132,3
2021-10-07 07:30:00,54547.1,54734.8,54532.5,54652.5,54528.066667,54763.790,53891.315,14.946098,1.584015,0.371707,1,0,0,3,254.576776,54277.923224,54989.376776,53978.746449,55401.653551,3
2021-10-07 07:45:00,54653.7,54747.6,54521.7,54725.5,54533.783333,54748.630,53923.689,14.616581,1.521458,0.472232,1,0,0,3,252.528435,54269.171565,55000.128435,54176.243131,55638.156869,3
2021-10-07 08:00:00,54724.3,54738.0,54454.3,54456.2,54526.800000,54722.685,53954.590,14.611427,1.540293,0.183844,1,0,0,3,254.754975,54199.545025,54992.754975,53383.090050,54969.509950,3
2021-10-07 08:15:00,54456.2,54589.9,54425.5,54429.6,54517.016667,54703.680,53986.213,14.735778,1.601323,0.187117,1,0,0,3,248.301048,54177.198952,54838.201048,53612.397904,54934.402096,3
2021-10-07 08:30:00,54430.0,54554.1,54365.0,54526.5,54556.266667,54684.145,54017.600,15.124152,1.583561,0.317953,1,0,0,3,244.072402,54120.927598,54798.172402,53983.155196,55337.644804,3


In [173]:
df['trade_result'].value_counts()

,count
trade_result,
3,58594
2,698
1,609


In [174]:
(df['valid_cross']==1).sum(), (df['valid_cross']==2).sum(), (df['valid_cross']==3).sum()

(1969, 1969, 55963)

In [175]:
total_concluded_trades = ((df['valid_cross'] == 1) | (df['valid_cross'] == 2)).sum()
winning_trades = (df['trade_result'] == 1).sum() + (df['trade_result'] == 2).sum()
win_rate = winning_trades / total_concluded_trades
print(f"Total trade signal: {total_concluded_trades}")
# print(f"Winning trades: {winning_trades}")
print(f"Accuracy: {win_rate * 100}")

Total trade signal: 3938
Accuracy: 33.18943626206196


In [176]:
## Now using machine learning, how can we improve these results
ml = df[(df['valid_cross'] == 1) | (df['valid_cross'] == 2)]
# print len of ml
print(len(ml))
y = ml['trade_result']  # Target variable
x = ml.drop(columns=['trade_result'])  # Features: drop the target column

3938


In [177]:
x

,Open,High,Low,Close,sma_6,sma_20,sma_100,adx,bb_width,bb_pband,trend,crossover,crossunder,valid_cross,ATR,LowerStopLoss,UpperStopLoss,LowerTakeProfit,UpperTakeProfit
Date,,,,,,,,,,,,,,,,,,,
2021-10-07 13:45:00,54619.0,54631.2,54342.4,54526.1,54185.450000,54105.055,54628.383,30.697829,1.565515,0.997088,2,1,0,1,312.275926,54030.124074,54943.475926,53691.348148,55518.051852
2021-10-07 15:15:00,53812.8,54154.1,53809.9,53901.5,54036.900000,54075.155,54616.234,28.943762,1.942837,0.334708,2,0,1,2,358.316093,53451.583907,54512.416093,52679.667814,54801.332186
2021-10-07 17:30:00,54113.2,54194.4,53987.1,54181.9,54090.283333,54055.130,54561.038,26.036235,2.009436,0.616709,2,1,0,1,319.148594,53667.951406,54513.548594,53518.602812,55209.797188
2021-10-07 18:45:00,53792.7,53925.2,53792.7,53849.4,53989.983333,53994.975,54527.579,25.403096,1.611239,0.332670,2,0,1,2,276.904274,53515.795726,54202.104274,53143.991452,54516.608548
2021-10-07 20:15:00,53888.8,54219.4,53875.9,54178.6,54000.466667,53984.045,54482.738,21.715609,1.092973,0.829737,2,1,0,1,249.716392,53626.183608,54469.116392,53597.567215,55283.432785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-09 14:30:00,30213.1,30407.2,30213.0,30302.5,30254.000000,30257.715,30217.161,18.114392,0.401234,0.868891,1,0,1,2,49.363013,30163.636987,30456.563013,29994.373974,30580.226026
2023-07-09 15:15:00,30285.8,30344.1,30257.2,30321.8,30271.233333,30266.435,30221.910,19.486655,0.464174,0.894088,1,1,0,1,56.732085,30200.467915,30400.832085,30163.735831,30564.464169
2023-07-09 17:15:00,30247.9,30254.4,30229.3,30244.1,30267.583333,30280.485,30232.179,16.247481,0.480530,0.249943,1,0,1,2,46.061704,30183.238296,30300.461704,30131.376592,30365.823408


In [178]:
x.shape, y.shape

((3938, 19), (3938,))

In [179]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# Initialize the RandomForestClassifier
model = RandomForestClassifier(n_estimators=1000, random_state=42)
# Train the model
model.fit(x_train, y_train)
# Make predictions
y_pred = model.predict(x_test)
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.63


In [180]:
## Testing on External data
test = pd.read_csv('BTCUSD_M15_TEST.csv')

In [181]:
test

,Date,Open,High,Low,Close,Vol
0,10/07/2023 09:30,30145.1,30166.3,30086.1,30112.0,1
1,10/07/2023 09:45,30112.2,30137.6,30112.0,30115.7,1
2,10/07/2023 10:00,30115.7,30138.6,30112.8,30134.9,1
3,10/07/2023 10:15,30134.8,30149.5,30133.6,30142.4,1
4,10/07/2023 10:30,30142.9,30174.6,30142.9,30167.2,1
...,...,...,...,...,...,...
39995,02/09/2024 12:45,58247.2,58386.2,58232.2,58356.1,1
39996,02/09/2024 13:00,58355.2,58400.2,58260.2,58375.2,1
39997,02/09/2024 13:15,58375.2,58455.2,58208.0,58235.5,1
39998,02/09/2024 13:30,58227.8,58284.9,58116.1,58197.3,1


In [182]:
df

,Open,High,Low,Close,sma_6,sma_20,sma_100,adx,bb_width,bb_pband,trend,crossover,crossunder,valid_cross,ATR,LowerStopLoss,UpperStopLoss,LowerTakeProfit,UpperTakeProfit,trade_result
Date,,,,,,,,,,,,,,,,,,,,
2021-10-07 06:30:00,54691.4,54691.4,54492.9,54498.1,54667.016667,54848.090,53773.157,13.196027,1.248902,-0.010935,1,0,0,3,250.025805,54242.874195,54941.425805,53611.448390,55008.551610,3
2021-10-07 06:45:00,54496.9,54622.4,54364.4,54488.3,54630.333333,54835.450,53803.218,13.608975,1.365987,0.036543,1,0,0,3,250.595390,54113.804610,54872.995390,53718.909219,55237.290781,3
2021-10-07 07:00:00,54488.3,54552.6,54228.2,54291.0,54575.150000,54805.570,53831.156,14.481079,1.613190,-0.082015,1,0,0,3,255.867148,53972.332852,54808.467148,53256.065704,54928.334296,3
2021-10-07 07:15:00,54287.7,54563.4,54269.3,54547.3,54551.333333,54780.600,53860.645,15.226554,1.610692,0.235592,1,0,0,3,258.598066,54010.701934,54821.998066,53997.903868,55620.496132,3
2021-10-07 07:30:00,54547.1,54734.8,54532.5,54652.5,54528.066667,54763.790,53891.315,14.946098,1.584015,0.371707,1,0,0,3,254.576776,54277.923224,54989.376776,53978.746449,55401.653551,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-10 08:15:00,30058.4,30078.1,29974.2,30068.6,30077.533333,30084.255,30181.739,17.128162,0.364366,0.357184,2,0,0,3,49.430416,29924.769584,30127.530416,29950.739169,30356.260831,3
2023-07-10 08:30:00,30068.4,30084.6,30047.9,30080.7,30072.750000,30081.260,30180.220,18.188795,0.321320,0.494206,2,0,0,3,48.521100,29999.378900,30133.121100,29975.857800,30243.342200,3
2023-07-10 08:45:00,30081.1,30082.9,30041.2,30069.9,30073.950000,30078.110,30178.539,19.303887,0.281200,0.402932,2,0,0,3,48.033879,29993.166121,30130.933879,29947.832242,30223.367758,3


In [183]:
test = data_cleaning(test)
test = evaluate_trades_precisely(test)
test = test[(test['valid_cross'] == 1) | (test['valid_cross'] == 2)]
test_data = test.drop(columns=['trade_result'])
test

,Open,High,Low,Close,sma_6,sma_20,sma_100,adx,bb_width,bb_pband,trend,crossover,crossunder,valid_cross,ATR,LowerStopLoss,UpperStopLoss,LowerTakeProfit,UpperTakeProfit,trade_result
Date,,,,,,,,,,,,,,,,,,,,
2023-07-11 14:00:00,30414.9,30464.0,30363.4,30379.7,30384.566667,30382.305,30420.432,18.656291,0.233666,0.463306,2,1,0,1,56.484761,30306.915239,30520.484761,30098.130478,30525.269522,1
2023-07-11 18:45:00,30454.6,30474.3,30422.0,30455.2,30509.616667,30531.035,30474.810,24.159222,0.814996,0.195230,2,0,1,2,74.623322,30347.376678,30548.923322,30267.753355,30670.846645,3
2023-07-11 20:30:00,30564.5,30565.7,30523.5,30539.7,30546.816667,30541.170,30480.413,19.846914,0.663608,0.492747,1,1,0,1,63.421587,30460.078413,30629.121587,30360.856826,30698.943174,1
2023-07-11 21:30:00,30498.9,30498.9,30483.2,30497.4,30525.566667,30535.145,30467.681,16.248491,0.669950,0.315491,1,0,1,2,54.660382,30428.539618,30553.560382,30385.079235,30635.120765,3
2023-07-11 22:45:00,30500.8,30557.5,30496.6,30551.0,30523.133333,30517.675,30463.751,15.537892,0.464909,0.734883,1,1,0,1,53.363553,30443.236447,30610.863553,30431.272894,30766.527106,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-01 21:45:00,58192.3,58319.6,58138.1,58279.5,58275.366667,58298.815,58337.110,17.627587,1.331289,0.475114,2,0,1,2,197.314090,57940.785910,58516.914090,57804.671820,58956.928180,2
2024-09-02 03:00:00,57466.0,57680.3,57459.9,57622.5,57462.683333,57394.825,58023.335,16.021790,0.686906,1.077491,2,1,0,1,248.739859,57211.160141,57929.039859,57009.420282,58445.179718,1
2024-09-02 07:45:00,57418.2,57529.5,57354.1,57512.2,57587.166667,57610.175,57887.494,12.795291,0.755543,0.274910,2,0,1,2,183.630497,57170.469503,57713.130497,57110.339006,58195.660994,3


In [184]:
test_data

,Open,High,Low,Close,sma_6,sma_20,sma_100,adx,bb_width,bb_pband,trend,crossover,crossunder,valid_cross,ATR,LowerStopLoss,UpperStopLoss,LowerTakeProfit,UpperTakeProfit
Date,,,,,,,,,,,,,,,,,,,
2023-07-11 14:00:00,30414.9,30464.0,30363.4,30379.7,30384.566667,30382.305,30420.432,18.656291,0.233666,0.463306,2,1,0,1,56.484761,30306.915239,30520.484761,30098.130478,30525.269522
2023-07-11 18:45:00,30454.6,30474.3,30422.0,30455.2,30509.616667,30531.035,30474.810,24.159222,0.814996,0.195230,2,0,1,2,74.623322,30347.376678,30548.923322,30267.753355,30670.846645
2023-07-11 20:30:00,30564.5,30565.7,30523.5,30539.7,30546.816667,30541.170,30480.413,19.846914,0.663608,0.492747,1,1,0,1,63.421587,30460.078413,30629.121587,30360.856826,30698.943174
2023-07-11 21:30:00,30498.9,30498.9,30483.2,30497.4,30525.566667,30535.145,30467.681,16.248491,0.669950,0.315491,1,0,1,2,54.660382,30428.539618,30553.560382,30385.079235,30635.120765
2023-07-11 22:45:00,30500.8,30557.5,30496.6,30551.0,30523.133333,30517.675,30463.751,15.537892,0.464909,0.734883,1,1,0,1,53.363553,30443.236447,30610.863553,30431.272894,30766.527106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-01 21:45:00,58192.3,58319.6,58138.1,58279.5,58275.366667,58298.815,58337.110,17.627587,1.331289,0.475114,2,0,1,2,197.314090,57940.785910,58516.914090,57804.671820,58956.928180
2024-09-02 03:00:00,57466.0,57680.3,57459.9,57622.5,57462.683333,57394.825,58023.335,16.021790,0.686906,1.077491,2,1,0,1,248.739859,57211.160141,57929.039859,57009.420282,58445.179718
2024-09-02 07:45:00,57418.2,57529.5,57354.1,57512.2,57587.166667,57610.175,57887.494,12.795291,0.755543,0.274910,2,0,1,2,183.630497,57170.469503,57713.130497,57110.339006,58195.660994


In [185]:
new_y = test['trade_result']
new_y

,trade_result
Date,
2023-07-11 14:00:00,1
2023-07-11 18:45:00,3
2023-07-11 20:30:00,1
2023-07-11 21:30:00,3
2023-07-11 22:45:00,1
...,...
2024-09-01 21:45:00,2
2024-09-02 03:00:00,1
2024-09-02 07:45:00,3


In [186]:
new_pred_y = model.predict(test_data)
new_pred_y

array([3, 3, 3, ..., 3, 3, 3])

In [187]:
accuracy = accuracy_score(new_y, new_pred_y)
print(f'Accuracy: {accuracy:.2f}')
print(len(new_pred_y))

Accuracy: 0.60
2548
